In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from clean_data import *
from hmm import DiscreteDistr, GaussDistr, GaussMixDistr

## Clean data and dimension reduction

In [2]:
# Prepare data for person 4. Use sequence 1~3 for training, 4~5 for testing.
person = 4
sadl_n = []
for n in range(1, 6):
    sadl_n.append(pd.read_table('data/S%d-ADL%d.dat' % (person, n), sep='\s+', header=None, dtype=float))

In [3]:
# Smooth data, time: col 0, features: col 1~36, labels: col 244 
winsize = 15
stepsize = 8

# train data
train_sample = np.empty((0, 36))
train_labels = np.empty((0))
train_len = []
for i in range(0, 3):
    print sadl_n[i].shape
    features = moving_avg(sadl_n[i].iloc[:, 1:37], winsize, stepsize)
    print features.shape
    labels = moving_vote_majority(sadl_n[i].iloc[:, 244], winsize, stepsize)
    train_sample = np.concatenate((train_sample, features), axis=0)
    train_len.append(features.shape[0])
    train_labels = np.concatenate( (train_labels, labels) )
train_len = np.array(train_len)

print "train_sample size: ", train_sample.shape
print "train_labels size: ", train_labels.shape
print "subsequence length: ", train_len, ". Sum of length: ", np.sum(train_len)

# test data
test_sample = np.empty((0, 36))
test_labels = np.empty((0))
test_len = []
for i in range(3, 5):
    features = moving_avg(sadl_n[i].iloc[:, 1:37], winsize, stepsize)
    labels = moving_vote_majority(sadl_n[i].iloc[:, 244], winsize, stepsize)
    test_sample = np.concatenate((test_sample, features), axis=0)
    test_len.append(features.shape[0])
    test_labels = np.concatenate( (test_labels, labels) )
test_len = np.array(test_len)  

print "test_sample size: ", test_sample.shape
print "test_labels size: ", test_labels.shape
print "subsequence length: ", test_len, ". Sum of length: ", np.sum(test_len)


(41588, 250)
(5198, 36)
(27737, 250)
(3467, 36)
(25132, 250)
(3141, 36)
train_sample size:  (11806, 36)
train_labels size:  (11806,)
subsequence length:  [5198 3467 3141] . Sum of length:  11806
test_sample size:  (6593, 36)
test_labels size:  (6593,)
subsequence length:  [2778 3815] . Sum of length:  6593


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:598: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


In [4]:
# Fill missing values
col_threshold = 0.5
train, test = fill_missing(train_sample, test_sample, col_threshold, True)
np.any(np.isnan(train)), np.any(np.isnan(test))
print train.shape

(11806, 36)


In [ ]:
# Normalize feature

In [ ]:
# Search for number of mixtures for each class

In [17]:
n_states = 6
# Train output distribution
train_label_transfer = np.maximum(0, train_labels - 100)
outputdistr_gmm = []
n_mix = [2, 1, 4, 2, 6, 4]
# n_mix = [1,1,1,1,1,1]
for i in range(0, n_states):
    gmm = GaussMixDistr(gauss=n_mix[i])
    gmm.init_by_data(train[train_label_transfer == i, :])
    gmm.train(train[train_label_transfer == i, :])
    outputdistr_gmm.append(gmm)

hmm/gauss_distr.py:243: UserWarning: Not enough data for GaussDistr object 2, forcing std to zero
  % i )
hmm/gauss_distr.py:243: UserWarning: Not enough data for GaussDistr object 5, forcing std to zero
  % i )
hmm/gauss_distr.py:243: UserWarning: Not enough data for GaussDistr object 3, forcing std to zero
  % i )


In [21]:
# revisit gmm model, check logprob
i = 2
logp_x = outputdistr_gmm[i].logprob(train[train_label_transfer == i, :])
np.mean(logp_x)

-242.0357135211238

In [15]:
j = 0
logp_xy = outputdistr_gmm[i].logprob(train[train_label_transfer == j, :])
np.mean(logp_xy)

-262.6729954880571